In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df = pd.read_csv('data/cs-training_ver02.csv')
x = df.drop(columns='seriousdlqin2yrs')
y = df.seriousdlqin2yrs

In [3]:
with open('xgb.pkl','rb') as f:
    data = joblib.load(f)
print(data.keys())
locals().update(data)

dict_keys(['xgb_best', 'xgb_param'])


In [4]:
xgb_param

{'n_estimators': 125, 'max_depth': 5, 'learning_rate': 0.04}

In [5]:
y.value_counts()

0.0    139974
1.0     10026
Name: seriousdlqin2yrs, dtype: int64

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier
new_model = XGBClassifier(**xgb_param)

In [7]:
cv = 10
for i in range(10):
    x_target = x.loc[y[y==1].index]
    x_nontarget = x.loc[y[y!=1].index].sample(len(x_target))
    x_sample = pd.concat([x_target,x_nontarget])
    y_new = np.concatenate([np.array([1]*len(x_target)), np.array([0]*len(x_target))])
    new_model.fit(x_sample, y_new)

In [11]:
from sklearn.model_selection import cross_val_score
# print(cross_val_score(new_model, x, y, cv=4, scoring='roc_auc').mean())
# print(cross_val_score(xgb_best, x, y, cv=4, scoring='roc_auc').mean())

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_over, y_over = smote.fit_sample(x,y)

In [9]:
new_model02 = XGBClassifier(**xgb_param)
new_model02.fit(x_over, y_over)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.04, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=125, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
print(cross_val_score(new_model02, x, y, cv=4, scoring='roc_auc').mean())
print(cross_val_score(xgb_best, x, y, cv=4, scoring='roc_auc').mean())

0.8616489423185827
0.8633861014395722
